In [1]:
import scanpy as sc
import numpy as np
import functools
import jax
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import scanpy as sc
import gseapy as gp
import matplotlib.pyplot as plt
import gseapy as gp
import anndata as ad
import decoupler
from pathlib import Path

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
split = 2

In [3]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [4]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [5]:
adata_ood.obs.head()

,sample,Size_Factor,n.umi,RT_well,Drug1,Drug2,Well,n_genes,n_genes_by_counts,total_counts,...,split,control,cell_type,cell_line,smiles_drug_1,smiles_drug_2,ood_1,ood_2,ood_3,ood_4
Cell,,,,,,,,,,,,,,,,,,,,,
A01_A02_RT_BC_10_Lig_BC_18,sciPlex_theis,0.533816,1433,RT_10,control,Panobinostat,A10,1004,1004,1433.0,...,ood,0,A549,A549,NaN,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)/C=C/C(=O)NO,not ood,control+Panobinostat,not ood,not ood
A01_A02_RT_BC_10_Lig_BC_25,sciPlex_theis,0.539404,1448,RT_10,control,Panobinostat,A10,1062,1062,1448.0,...,ood,0,A549,A549,NaN,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)/C=C/C(=O)NO,not ood,control+Panobinostat,not ood,not ood
A01_A02_RT_BC_10_Lig_BC_48,sciPlex_theis,0.532326,1429,RT_10,control,Panobinostat,A10,1063,1063,1429.0,...,ood,0,A549,A549,NaN,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)/C=C/C(=O)NO,not ood,control+Panobinostat,not ood,not ood
A01_A02_RT_BC_10_Lig_BC_51,sciPlex_theis,1.461754,3924,RT_10,control,Panobinostat,A10,2412,2411,3923.0,...,ood,0,A549,A549,NaN,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)/C=C/C(=O)NO,not ood,control+Panobinostat,not ood,not ood
A01_A02_RT_BC_10_Lig_BC_70,sciPlex_theis,0.418708,1124,RT_10,control,Panobinostat,A10,834,832,1122.0,...,ood,0,A549,A549,NaN,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)/C=C/C(=O)NO,not ood,control+Panobinostat,not ood,not ood


In [6]:
adata_ood.obs["condition"].cat.categories

Index(['Dacinostat+Dasatinib', 'Dacinostat+PCI-34051', 'Givinostat+Cediranib',
       'Givinostat+Curcumin', 'Panobinostat+Alvespimycin',
       'Panobinostat+SRT3025', 'control', 'control+Panobinostat'],
      dtype='object')

In [7]:
adata_train.obs["condition"].cat.categories

Index(['Alvespimycin+Pirarubicin', 'Cediranib+PCI-34051',
       'Dacinostat+Danusertib', 'Givinostat+Carmofur', 'Givinostat+Crizotinib',
       'Givinostat+Dasatinib', 'Givinostat+SRT1720', 'Givinostat+SRT2104',
       'Givinostat+Sorafenib', 'Givinostat+Tanespimycin',
       'Panobinostat+Crizotinib', 'Panobinostat+Curcumin',
       'Panobinostat+Dasatinib', 'Panobinostat+PCI-34051',
       'Panobinostat+SRT1720', 'Panobinostat+SRT2104',
       'Panobinostat+Sorafenib', 'SRT2104+Alvespimycin', 'SRT3025+Cediranib',
       'control', 'control+Alvespimycin', 'control+Dacinostat',
       'control+Dasatinib', 'control+Givinostat', 'control+SRT2104'],
      dtype='object')

In [8]:
control_mean = adata_ood[adata_ood.obs["condition"]=="control"].X.mean(axis=0)
rng = np.random.default_rng(0)
drug_effect = {}
for cond in adata_train.obs["condition"].cat.categories:
    if cond=="control":
        continue
    drug_effect[cond] = adata_train[adata_train.obs["condition"]==cond].X.mean(axis=0) - control_mean

drug_effect["control"] = adata_ood[adata_ood.obs["condition"]=="control"].X.toarray()
    

In [9]:
drug_effect_ood = {}

drug_effect_ood['Dacinostat+Dasatinib'] = (
    drug_effect['control'] 
    + drug_effect['control+Dacinostat'] 
    + drug_effect['control+Dasatinib']
)

drug_effect_ood['Dacinostat+PCI-34051'] = (
    drug_effect['control'] + 
    drug_effect['Panobinostat+PCI-34051'] - 
    drug_effect['Panobinostat+SRT2104'] -
    drug_effect['control+SRT2104'] +
    drug_effect['control+Dacinostat']
)

drug_effect_ood['Givinostat+Cediranib'] = (
    drug_effect['control'] + 
    drug_effect['Cediranib+PCI-34051'] 
    - drug_effect['Panobinostat+PCI-34051'] 
    + drug_effect['Panobinostat+SRT2104']
    - drug_effect['control+SRT2104']
    + drug_effect['control+Givinostat']
)

drug_effect_ood['Givinostat+Curcumin'] = (
    drug_effect['control'] + 
    + drug_effect['Panobinostat+Curcumin'] 
    - drug_effect['Panobinostat+SRT2104']
    + drug_effect['control+SRT2104']
    + drug_effect['control+Givinostat']
)

drug_effect_ood['Panobinostat+Alvespimycin'] = (
    drug_effect['control'] + 
    + drug_effect['control+Alvespimycin'] 
    + drug_effect['Panobinostat+SRT2104']
    - drug_effect['control+SRT2104']
)

drug_effect_ood['Panobinostat+SRT3025'] = (
    drug_effect['control'] + 
    + drug_effect['SRT3025+Cediranib'] 
    - drug_effect['Cediranib+PCI-34051'] 
    + drug_effect['Panobinostat+PCI-34051'] 
)

drug_effect_ood['control+Panobinostat'] = (
    drug_effect['control'] + 
    drug_effect['Panobinostat+SRT2104'] 
    - drug_effect['control+SRT2104']
)

In [10]:
import anndata as ad
import pandas as pd
all_data = []
conditions = []

for condition, array in drug_effect_ood.items():
    all_data.append(array)
    conditions.extend([condition] * array.shape[0])

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)

# Create a DataFrame for the .obs attribute
obs_data = pd.DataFrame({
    'condition': conditions
})

# Create the Anndata object
adata_pred_ood = ad.AnnData(X=all_data_array, obs=obs_data)

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [11]:
adata_pred_ood.X = np.array(adata_pred_ood.X)

In [12]:
adata_pred_ood.write(f"/lustre/groups/ml01/workspace/ot_perturbation/models/additive/combosciplex/adata_ood_with_predictions_{split}.h5ad")